In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import IPython.display as ipd
from pathlib import Path
from multiprocessing import Pool
import torch
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from data_loading_utils import *
from preprocessing_utils import *
from helpers import *
from metrics import *

In [3]:
PATH = Path('data/')
TRAIN_PATH = PATH/'audio_train_24000'
TEST_PATH = PATH/'audio_test_24000'

sample_rate = 24000
n_features = 128
n_fft = 1024
hop_length = 256

In [4]:
train = pd.read_csv(PATH/'train.csv')
test = pd.read_csv(PATH/'sample_submission.csv')
labels = sorted(train.label.unique())
label_idx = {label:i for i, label in enumerate(labels)}

In [5]:
train_fname = train.fname
test_fname = test.fname 

In [6]:
len(train_fname)

9473

In [ ]:
TRAIN_PATH_LMS = PATH/f'audio_train_tfms_{sample_rate}_{n_fft}_{hop_length}_{n_features}'
TRAIN_PATH_LMS.mkdir(exist_ok=True)
# TEST_PATH_LMS = PATH/f'audio_test_{sample_rate}_{n_fft}_{hop_length}_{n_features}'
# str(TRAIN_PATH_LMS), str(TEST_PATH_LMS)

In [23]:
trn_files = list(TRAIN_PATH_LMS.iterdir())
trn_features = [np.load(f) for f in trn_files]
len(trn_features)
# test_files = list(TEST_PATH_LMS.iterdir())
# test_features = [np.load(f) for f in test_files]
# len(trn_files), #len(test_files)



9473

In [24]:
def get_stats(objects):
    sizes, means, stds = zip(*((o.size, o.mean(), o.std()) for o in objects))
    sizes, means, stds = np.array(sizes), np.array(means), np.array(stds)
    weighted_mean = (means * sizes).sum() / sizes.sum()
    # https://stats.stackexchange.com/questions/55999/is-it-possible-to-find-the-combined-standard-deviation
    weighted_std = np.sqrt(((sizes * stds**2.0).sum() + (sizes * (means - weighted_mean)**2.0).sum())/(sizes.sum()))
    return weighted_mean, weighted_std

mean, std = get_stats(trn_features)
mean, std

(-34.1864115452616, 17.731700449450333)

### Calculating get_stats

In [ ]:
def convert_feature(filename, src_path=TRAIN_PATH, dst_path=TRAIN_PATH_LMS, sample_rate=sample_rate):
    data, file_sr = read_file(filename, path=src_path, sample_rate=sample_rate)
    if data.shape[0] > 0:
        mel_spec = librosa.feature.melspectrogram(data,
                                                  sr=sample_rate, 
                                                  n_mels=n_features, 
                                                  n_fft=n_fft, 
                                                  hop_length=hop_length)
    else:
        mel_spec = np.zeros((n_mels, 1))
    power_spec = librosa.power_to_db(mel_spec)
    dst_filename = dst_path/f'{filename}.npy'
    np.save(dst_filename, power_spec)

In [17]:
convert_stft(train_fname[0])

In [20]:
fnames = train_fname.values
len(fnames)

9473

In [ ]:
processes = 8

with Pool(processes=processes) as pool:
    total_tfms = len(fnames)    
    with tqdm(total=total_tfms) as pbar:
        for _ in pool.imap_unordered(convert_feature, fnames):
            pbar.update()

A Jupyter Widget